
# Pytorch + HuggingFace 
## KoElectra Model
박장원님의 KoElectra-small 사용<br>
https://monologg.kr/2020/05/02/koelectra-part1/<br>
https://github.com/monologg/KoELECTRA

## Dataset
네이버 영화 리뷰 데이터셋<br>
https://github.com/e9t/nsmc

## References
- https://huggingface.co/transformers/training.html
- https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
- https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html
- https://wikidocs.net/44249

## 주의사항
꼭 GPU로 해주세요 - 1epoch 당 약 20분 소요

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [ ]:
# GPU 사용
device = torch.device("cuda")

# Dataset 만들어서 불러오기 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_sent = pd.read_csv('/content/drive/MyDrive/뉴스원인분석_modeling/final_news_modeling/lgenergy/lgenergy_sent_split.csv')
df_sent = df_sent.sample(frac=1, random_state=2022).reset_index(drop=True)
df_gr = df_sent.groupby('date')[['target']].mean()
# 날짜별로 train_test_split
train_date, test_date = train_test_split(df_gr, test_size=0.1, random_state=2022, stratify=df_gr['target'])
train_date_list = train_date.index
test_date_list = test_date.index
train_data = df_sent[df_sent['date'].isin(train_date_list)][['date','sent', 'target','targetpm','news_id']]
test_data = df_sent[df_sent['date'].isin(test_date_list)][['date','sent', 'target','targetpm','news_id']]
train_data

,date,sent,target,targetpm,news_id
0,2022-08-05,"코스닥시장에서는 외국인이 576억원, 기관이 396억원을 순매수했다.",2,3,3900
1,2022-06-10,"지배구조 개편이라는 숙제를 안고 있는 현대차그룹의 인 서강현현대차, 주우정기아 부사...",2,3,2728
2,2022-08-29,"반기문 , 세계시민정신으로 협력실천 강화...31일까지 글로벌 포럼반기문사진 전 유...",1,1,4295
4,2022-02-17,은 전기차 대중화를 위한 두 모델 출시를 시작으로 2025년까지 30종의 전기차를 ...,2,1,616
6,2022-09-15,코스닥 지수는 전 거래일 대비 1.55포인트0.20 하락한 781.38에 거래됐다.,3,3,4568
...,...,...,...,...,...
125902,2022-06-07,"3월 말 현재 지난해 말 대비 직원 수를 각각 157명, 268명 늘렸다.",1,1,2652
125903,2022-07-12,화학의 뒤를 이어 내주 한국주택금융공사가 발행에 나설 것으로 관측된다.,3,3,3361
125905,2022-09-29,해법 전기차 현지생산 앞당기고 이차전지 공급망 다변화 나서야이데일리 김형욱 기자 한...,3,3,4839
125906,2022-02-24,유앤아이의 자회사 디엠파워는 2차전지 검사장비와 전기차 충전기 전문기업인 인피니티웍...,0,1,790


In [ ]:
train_data = train_data.iloc[:,1:3]
train_data = train_data.reset_index()
train_data

,index,sent,target
0,0,"코스닥시장에서는 외국인이 576억원, 기관이 396억원을 순매수했다.",2
1,1,"지배구조 개편이라는 숙제를 안고 있는 현대차그룹의 인 서강현현대차, 주우정기아 부사...",2
2,2,"반기문 , 세계시민정신으로 협력실천 강화...31일까지 글로벌 포럼반기문사진 전 유...",1
3,4,은 전기차 대중화를 위한 두 모델 출시를 시작으로 2025년까지 30종의 전기차를 ...,2
4,6,코스닥 지수는 전 거래일 대비 1.55포인트0.20 하락한 781.38에 거래됐다.,3
...,...,...,...
113182,125902,"3월 말 현재 지난해 말 대비 직원 수를 각각 157명, 268명 늘렸다.",1
113183,125903,화학의 뒤를 이어 내주 한국주택금융공사가 발행에 나설 것으로 관측된다.,3
113184,125905,해법 전기차 현지생산 앞당기고 이차전지 공급망 다변화 나서야이데일리 김형욱 기자 한...,3
113185,125906,유앤아이의 자회사 디엠파워는 2차전지 검사장비와 전기차 충전기 전문기업인 인피니티웍...,0


In [ ]:
test_data = test_data.iloc[:,1:3]
test_data = test_data.reset_index()
test_data

,index,sent,target
0,3,한지영 키움증권 연구원은 국제원자력기구 사무총장의 북한 풍계리 연쇄 핵실험 가능성을...,0
1,5,업계 관계자는 수년간 저금리 속에 사업을 대폭 확장해온 바이오플랫폼 업종이 금리 상...,0
2,7,필라델피아 반도체 지수는 미 상원이 반도체 지원 법안에 절차적 표결을 통과시키면서 ...,3
3,8,"에너지솔루션, 글로벌 86개 파트너와 품질 경쟁력 논의헤럴드경제원호연 기자 에너지솔...",3
4,19,올해 경제개발협력기구 자료에 따르면 한국의 연구개발 인력은 인력 수 기준 세계 5위...,2
...,...,...,...
12717,125844,김현수 하나금융투자 연구원은 반도체 공급 부족 및 러시아우크라이나 전쟁에 따른 중대...,2
12718,125881,그는 2017년 초 화학이 생명과학을 흡수합병하며 신설한 생명과학사업본부의 수장부사...,2
12719,125895,정원석 하이투자증권 연구원은 테슬라의 차세대 4680 배터리에 건식 전극 공정이 적...,2
12720,125904,"화학051910, 삼성전자005930 등도 1 미만 약세다.",1


In [ ]:
class NSMCDataset(Dataset):
  
  def __init__(self, df):
    # 일부 값중에 NaN이 있음...
    self.dataset = df
    # 중복제거
    # self.dataset.drop_duplicates(subset=['document'], inplace=True)
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 1:3].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [ ]:
train_dataset = NSMCDataset(train_data)
test_dataset = NSMCDataset(test_data)

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/486 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255k [00:00<?, ?B/s]

               index         target
count  113187.000000  113187.000000
mean    62986.182238       2.018589
std     36321.440684       1.290130
min         0.000000       0.000000
25%     31556.500000       1.000000
50%     63049.000000       2.000000
75%     94402.500000       3.000000
max    125907.000000       4.000000
               index        target
count   12722.000000  12722.000000
mean    62667.676230      1.934523
std     36573.230366      1.273361
min         3.000000      0.000000
25%     30702.250000      1.000000
50%     62181.500000      2.000000
75%     94586.750000      3.000000
max    125908.000000      4.000000


# Create Model

In [ ]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=5).to(device)

# 한번 실행해보기
# text, attention_mask, y = train_dataset[0]
# model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

# Learn

In [ ]:
epochs = 20
batch_size = 64

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

  0%|          | 0/1769 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Batch Loss: 159.52546548843384 Accuracy: tensor(0.2302, device='cuda:0')
Batch Loss: 318.65391516685486 Accuracy: tensor(0.2323, device='cuda:0')
Batch Loss: 477.4639879465103 Accuracy: tensor(0.2359, device='cuda:0')
Batch Loss: 636.2363342046738 Accuracy: tensor(0.2398, device='cuda:0')
Batch Loss: 794.7712279558182 Accuracy: tensor(0.2405, device='cuda:0')
Batch Loss: 953.3539533615112 Accuracy: tensor(0.2423, device='cuda:0')
Batch Loss: 1112.3673639297485 Accuracy: tensor(0.2423, device='cuda:0')
Batch Loss: 1271.5874027013779 Accuracy: tensor(0.2407, device='cuda:0')
Batch Loss: 1430.4599009752274 Accuracy: tensor(0.2409, device='cuda:0')
Batch Loss: 1589.647116303444 Accuracy: tensor(0.2405, device='cuda:0')
Batch Loss: 1748.6199414730072 Accuracy: tensor(0.2406, device='cuda:0')
Batch Loss: 1907.3473685979843 Accuracy: tensor(0.2407, device='cuda:0')
Batch Loss: 2066.081899523735 Accuracy: tensor(0.2414, device='cuda:0')
Batch Loss: 2224.590122461319 Accuracy: tensor(0.2416, de

  0%|          | 0/1769 [00:00<?, ?it/s]

Batch Loss: 158.4848359823227 Accuracy: tensor(0.2344, device='cuda:0')
Batch Loss: 317.40478575229645 Accuracy: tensor(0.2355, device='cuda:0')
Batch Loss: 476.04007387161255 Accuracy: tensor(0.2417, device='cuda:0')
Batch Loss: 634.4984992742538 Accuracy: tensor(0.2454, device='cuda:0')
Batch Loss: 793.2676032781601 Accuracy: tensor(0.2454, device='cuda:0')
Batch Loss: 951.9278898239136 Accuracy: tensor(0.2457, device='cuda:0')
Batch Loss: 1110.6627869606018 Accuracy: tensor(0.2458, device='cuda:0')
Batch Loss: 1269.2089282274246 Accuracy: tensor(0.2460, device='cuda:0')
Batch Loss: 1427.3817620277405 Accuracy: tensor(0.2472, device='cuda:0')
Batch Loss: 1585.3559186458588 Accuracy: tensor(0.2477, device='cuda:0')
Batch Loss: 1744.0034631490707 Accuracy: tensor(0.2481, device='cuda:0')
Batch Loss: 1902.2322540283203 Accuracy: tensor(0.2479, device='cuda:0')
Batch Loss: 2060.0185478925705 Accuracy: tensor(0.2488, device='cuda:0')
Batch Loss: 2218.6460115909576 Accuracy: tensor(0.2489,

...

Batch Loss: 115.42873442173004 Accuracy: tensor(0.5431, device='cuda:0')
Batch Loss: 230.0899065732956 Accuracy: tensor(0.5479, device='cuda:0')
Batch Loss: 344.34115850925446 Accuracy: tensor(0.5479, device='cuda:0')
Batch Loss: 459.3537520170212 Accuracy: tensor(0.5475, device='cuda:0')
Batch Loss: 575.3598597049713 Accuracy: tensor(0.5442, device='cuda:0')
Batch Loss: 691.15904545784 Accuracy: tensor(0.5438, device='cuda:0')
Batch Loss: 808.0752458572388 Accuracy: tensor(0.5429, device='cuda:0')
Batch Loss: 921.3518217206001 Accuracy: tensor(0.5446, device='cuda:0')
Batch Loss: 1037.709698021412 Accuracy: tensor(0.5440, device='cuda:0')
Batch Loss: 1154.4459403157234 Accuracy: tensor(0.5433, device='cuda:0')
Batch Loss: 1269.8377531766891 Accuracy: tensor(0.5431, device='cuda:0')
Batch Loss: 1385.0676972270012 Accuracy: tensor(0.5431, device='cuda:0')
Batch Loss: 1501.8312816023827 Accuracy: tensor(0.5426, device='cuda:0')
Batch Loss: 1617.1135199069977 Accuracy: tensor(0.5429, devi

  0%|          | 0/1769 [00:00<?, ?it/s]

Batch Loss: 111.44746214151382 Accuracy: tensor(0.5625, device='cuda:0')
Batch Loss: 224.3446902036667 Accuracy: tensor(0.5598, device='cuda:0')
Batch Loss: 336.55869483947754 Accuracy: tensor(0.5601, device='cuda:0')
Batch Loss: 449.09500563144684 Accuracy: tensor(0.5568, device='cuda:0')
Batch Loss: 561.9021844267845 Accuracy: tensor(0.5563, device='cuda:0')
Batch Loss: 673.0577870607376 Accuracy: tensor(0.5576, device='cuda:0')
Batch Loss: 787.5259730219841 Accuracy: tensor(0.5558, device='cuda:0')
Batch Loss: 900.639629304409 Accuracy: tensor(0.5561, device='cuda:0')
Batch Loss: 1012.7922018766403 Accuracy: tensor(0.5555, device='cuda:0')
Batch Loss: 1125.9413455724716 Accuracy: tensor(0.5548, device='cuda:0')
Batch Loss: 1238.0433509349823 Accuracy: tensor(0.5549, device='cuda:0')
Batch Loss: 1352.025168955326 Accuracy: tensor(0.5541, device='cuda:0')
Batch Loss: 1465.229078233242 Accuracy: tensor(0.5539, device='cuda:0')
Batch Loss: 1577.5942232012749 Accuracy: tensor(0.5543, dev

  0%|          | 0/1769 [00:00<?, ?it/s]

Batch Loss: 109.55338138341904 Accuracy: tensor(0.5702, device='cuda:0')
Batch Loss: 220.356227517128 Accuracy: tensor(0.5670, device='cuda:0')
Batch Loss: 332.541453063488 Accuracy: tensor(0.5635, device='cuda:0')
Batch Loss: 442.9279245734215 Accuracy: tensor(0.5643, device='cuda:0')
Batch Loss: 553.3571031689644 Accuracy: tensor(0.5651, device='cuda:0')
Batch Loss: 664.2651064991951 Accuracy: tensor(0.5652, device='cuda:0')
Batch Loss: 774.0543215274811 Accuracy: tensor(0.5660, device='cuda:0')
Batch Loss: 881.930652141571 Accuracy: tensor(0.5676, device='cuda:0')
Batch Loss: 990.0635166168213 Accuracy: tensor(0.5684, device='cuda:0')
Batch Loss: 1098.1353623867035 Accuracy: tensor(0.5690, device='cuda:0')
Batch Loss: 1207.6218904852867 Accuracy: tensor(0.5695, device='cuda:0')
Batch Loss: 1317.2173628807068 Accuracy: tensor(0.5694, device='cuda:0')
Batch Loss: 1426.3463526368141 Accuracy: tensor(0.5690, device='cuda:0')
Batch Loss: 1534.905876994133 Accuracy: tensor(0.5693, device=

In [ ]:
torch.save(model, '/content/drive/MyDrive/뉴스원인분석_modeling/model/KoELECTRA_lgenergy_0112.pt')

In [ ]:
model = torch.load('/content/drive/MyDrive/뉴스원인분석_modeling/model/KoELECTRA_lgenergy_0112.pt')
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
def board_pred(model, test):
  model.eval()
  from transformers import TextClassificationPipeline
  sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

  for idx in tqdm(test.index) :
    review_list = [test.loc[idx,'sent']]
    for review in review_list:
      pred = sentiment_classifier(review[:128])
      pred_label = pred[0]['label'][-1]
      pred_score = pred[0]['score']
      test.loc[idx,'pred'] = pred_label
      test.loc[idx,'score'] = pred_score
  return test

In [ ]:
def eval_model(test_pred) :
  test_pred['pred'] = test_pred['pred'].astype('int')
  # 뉴스별 라벨링
  df_news = test_pred.groupby('news_id')[['target', 'pred', 'targetpm']].mean()
  df_news_idx = df_news.index

  # 뉴스별 최빈값 계산
  for idx in df_news_idx:
    df_news.loc[idx, 'cnt_pred'] = int(test_pred[test_pred['news_id'] == idx ]['pred'].value_counts().idxmax())
    # print(idx, df_news.loc[idx, 'cnt_pred'])
    
  # 뉴스별 값 태깅
  test_pred['news_pred'] = -5
  for idx_nm in range(len(test_pred)) :
    news_idx = test_pred.iloc[idx_nm, 4]
    test_pred.iloc[idx_nm, -1] = df_news.loc[news_idx, 'cnt_pred']

  # 날짜별 최반값 계산
  df_gr = test_pred.groupby('date')[['target', 'news_pred', 'targetpm']].mean()
  df_date = df_gr.index
  for idx in df_date:
    df_gr.loc[idx, 'cnt_pred'] = int(test_pred[test_pred['date'] == idx ]['news_pred'].value_counts().idxmax())

  df_gr['pred_target'] = df_gr['cnt_pred'].apply(lambda x : int(np.round(x)))
  df_gr['pred_pm'] = df_gr['pred_target'].apply(lambda x: 1 if x< 2 else 3)

  df_gr = df_gr.reset_index()
  for idx in df_gr.index :
    if df_gr.loc[idx, 'target'] == df_gr.loc[idx, 'pred_target']:
      temp_eval = True
    else :
      temp_eval = False
    df_gr.loc[idx, 'eval_5'] = temp_eval

  for idx in df_gr.index :
    if df_gr.loc[idx, 'target'] == 0 :
      df_gr.loc[idx, '3target'] = 1
    elif df_gr.loc[idx, 'target'] == 4 :
      df_gr.loc[idx, '3target'] = 3
    else :
      df_gr.loc[idx, '3target'] = int(df_gr.loc[idx, 'target'])

    if df_gr.loc[idx, 'pred_target'] == 0 :
      df_gr.loc[idx, 'pred_3target'] = 1
    elif df_gr.loc[idx, 'pred_target'] == 4 :
      df_gr.loc[idx, 'pred_3target'] = 3
    else :
      df_gr.loc[idx, 'pred_3target'] = int(df_gr.loc[idx, 'pred_target'])

  df_gr['pred_3target'] = df_gr['pred_3target'].apply(lambda x : int(x))

  for idx in df_gr.index :
    if df_gr.loc[idx, '3target'] == df_gr.loc[idx, 'pred_3target']:
      temp_eval = True
    else :
      temp_eval = False
    df_gr.loc[idx, 'eval_3'] = temp_eval

  for idx in df_gr.index :
    if df_gr.loc[idx, 'targetpm'] == df_gr.loc[idx, 'pred_pm']:
      temp_eval = True
    else :
      temp_eval = False
    df_gr.loc[idx, 'eval_pm'] = temp_eval


  print('5중분류 정확도 :', round(df_gr['eval_5'].sum() / len(df_gr), 2), '\n')
  print('3중분류 정확도 :', round(df_gr['eval_3'].sum() / len(df_gr), 2), '\n')
  print('이진분류 정확도 :', round(df_gr['eval_pm'].sum() / len(df_gr), 2), '\n')

  df_gr.head()
  return test_pred, df_gr

In [ ]:
test_data.head()

,date,sent,target,targetpm,news_id
3,2022-06-20,한지영 키움증권 연구원은 국제원자력기구 사무총장의 북한 풍계리 연쇄 핵실험 가능성을...,0,1,2914
5,2022-07-01,업계 관계자는 수년간 저금리 속에 사업을 대폭 확장해온 바이오플랫폼 업종이 금리 상...,0,1,3154
7,2022-07-21,필라델피아 반도체 지수는 미 상원이 반도체 지원 법안에 절차적 표결을 통과시키면서 ...,3,3,3552
8,2022-02-16,"에너지솔루션, 글로벌 86개 파트너와 품질 경쟁력 논의헤럴드경제원호연 기자 에너지솔...",3,3,598
19,2022-07-14,올해 경제개발협력기구 자료에 따르면 한국의 연구개발 인력은 인력 수 기준 세계 5위...,2,1,3420


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")
model = model.cpu()
model.eval()
from transformers import TextClassificationPipeline
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

for idx in tqdm(test_data.index) :
  review_list = [test_data.loc[idx,'sent']]
  for review in review_list:
    pred = sentiment_classifier(review[:128])
    pred_label = pred[0]['label'][-1]
    pred_score = pred[0]['score']
    test_data.loc[idx,'pred'] = pred_label
    test_data.loc[idx,'score'] = pred_score
test_data

  0%|          | 0/12722 [00:00<?, ?it/s]

,date,sent,target,targetpm,news_id,pred,score
3,2022-06-20,한지영 키움증권 연구원은 국제원자력기구 사무총장의 북한 풍계리 연쇄 핵실험 가능성을...,0,1,2914,1,0.708904
5,2022-07-01,업계 관계자는 수년간 저금리 속에 사업을 대폭 확장해온 바이오플랫폼 업종이 금리 상...,0,1,3154,1,0.702289
7,2022-07-21,필라델피아 반도체 지수는 미 상원이 반도체 지원 법안에 절차적 표결을 통과시키면서 ...,3,3,3552,3,0.431405
8,2022-02-16,"에너지솔루션, 글로벌 86개 파트너와 품질 경쟁력 논의헤럴드경제원호연 기자 에너지솔...",3,3,598,0,0.445601
19,2022-07-14,올해 경제개발협력기구 자료에 따르면 한국의 연구개발 인력은 인력 수 기준 세계 5위...,2,1,3420,2,0.462013
...,...,...,...,...,...,...,...
125844,2022-03-30,김현수 하나금융투자 연구원은 반도체 공급 부족 및 러시아우크라이나 전쟁에 따른 중대...,2,1,1468,4,0.510887
125881,2022-10-20,그는 2017년 초 화학이 생명과학을 흡수합병하며 신설한 생명과학사업본부의 수장부사...,2,1,5184,2,0.733772
125895,2022-06-21,정원석 하이투자증권 연구원은 테슬라의 차세대 4680 배터리에 건식 전극 공정이 적...,2,3,2941,0,0.361339
125904,2022-05-09,"화학051910, 삼성전자005930 등도 1 미만 약세다.",1,1,2171,3,0.615098


In [ ]:
import numpy as np
test_sent, test_date = eval_model(test_data)

5중분류 정확도 : 0.29 

3중분류 정확도 : 0.48 

이진분류 정확도 : 0.62 



In [ ]:
test_sent.to_csv('/content/drive/MyDrive/뉴스원인분석_modeling/data/KoELECTRA_lgenergy.test_sent_1013.csv', index=False)
test_date.to_csv('/content/drive/MyDrive/뉴스원인분석_modeling/data/KoELECTRA_lgenergy.test_date_1013.csv', index=False)